In [7]:
import pandas as pd 
import numpy as np 
from Bio import SeqIO
from src.tools import NCBIDatasets
from src.files import FASTAFile

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# Changing my training dataset strategy. Instead going to use the sequences directly from NCBI. 
genome_metadata_df = pd.read_csv('../data/genome_metadata.csv', index_col=0)
genome_ids = genome_metadata_df[genome_metadata_df.phylum == 'Campylobacterota'].index.values 


In [6]:
ncbi = NCBIDatasets()
ncbi.run(genome_ids=genome_ids, metadata=False, include=['protein'], dirs={'protein':'../data/ncbi/proteins'})
ncbi.cleanup()

NCBIDatasets._get_genome: Downloading data for GCF_900197775.1.: 100%|██████████| 197/197 [00:00<00:00, 2492.14it/s]


In [9]:
FASTAFile(path='../data/ncbi/proteins/GCF_003245365.1_protein.faa').to_df(prodigal_output=False)

,description,seq
id,,
WP_002876473.1,MULTISPECIES: hypothetical protein [Campylobac...,MFFAGGDYEPFIYGLSLIAYYFLPLYCLTLLLYKYKPQYVNLTLHI...
WP_005020782.1,MULTISPECIES: 30S ribosomal protein S21 [Helic...,MPGIKVRENESFDDAYRKFKKQADRNLVVTESRARRFFEPMTEKRK...
WP_005021381.1,MULTISPECIES: 50S ribosomal protein L33 [Helic...,MAKGNRVKIGLKCSECGDINYSTVKNAKTQTEKLELKKFCPRLNKH...
WP_005022299.1,MULTISPECIES: 50S ribosomal protein L36 [Helic...,MKVRPSVKKMCDKCKVIKRKGVIRVICENPKHKQRQG
WP_006801843.1,MULTISPECIES: translation initiation factor IF...,MAKDDVIEVDGIVKEALPNATFRVELENGHIILCHIAGRMRMHYIK...
...,...,...
WP_342767091.1,AtpZ/AtpI family protein [Helicobacter valdivi...,MENEKVQNKAKEEIMENHNQVFMQKEAKEELKYKDALLAYSNATLG...
WP_342767092.1,restriction endonuclease subunit S [Helicobact...,MQQHNLTLPVYCHSEALPEESKENPARFFANTQNDKLDSIKDVSQP...
WP_342767093.1,galactosyltransferase-related protein [Helicob...,MLELINIKEIDKNPNAFLVLPCVFLNEFGTEIYLNQNFYNKDMLFQ...
